In [ ]:
import ast
import re
import math
import operator as op

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import Tool, create_tool_calling_agent, AgentExecutor
from langchain.tools import StructuredTool
from typing import List, Optional
from pydantic import BaseModel

load_dotenv()

In [ ]:
# --- Tools ---

# --- 1. Mathematical expressions ---
def math_expression_calculator(input: str) -> str:
    if re.search(r"x(\^2|²)?", input, re.IGNORECASE):
        return "Error: Detected variable 'x' or quadratic term. Use the QuadraticEquationSolver for equations with x."
    expr = input.replace(" ", "")
    if re.search(r"[a-zA-Z]", expr):
        return "Error: Only pure math expressions with numbers and operators are allowed."
    expr = re.sub(r"[^0-9\.\+\-\*\/\(\)]", "", expr)
    if not expr:
        return "Error: No valid math expression found in input."
    try:
        result = eval(expr, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
    
# --- 2. Date difference calculator ---
def days_passed_calculation(input: str) -> str:
    match = re.search(r"\d{4}-\d{2}-\d{2}", input)
    if not match:
        return "Error: No date in 'YYYY-MM-DD' format found in input."
    date_str = match.group()
    try:
        user_date = datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return f"Error: Could not parse '{date_str}' with format '%Y-%m-%d'."
    current_date = datetime.today()
    time_difference: timedelta = current_date - user_date
    days_passed = time_difference.days
    return f"{days_passed} days have passed since {user_date.date()}." # str(days_passed)

# --- 3. Fahrenheit to Celsius conversion tool ---
def fahrenheit_to_celsius(input: str) -> str:
    try:
        match = re.search(r"[-+]?\d*\.?\d+", input)
        if not match:
            return "Error: No valid number found."
        temp_in_fahrenheit = float(match.group())
        temp_in_celsius = (temp_in_fahrenheit - 32) * 5.0/9.0
        return f"{temp_in_fahrenheit}°F is {temp_in_celsius:.2f}°C."
    except ValueError:
        return "Error: Invalid input. Please provide a valid number."
    
# --- 4. Quadratic equation solver ---
def parse_coefficients(s: str) -> float:
    s = s.strip()
    if s in ["", "+"]:
        return 1.0
    if s == "-":
        return -1.0
    try:
        return float(s)
    except ValueError:
        raise ValueError(f"Invalid coefficient: {s}")

def quadratic_equation_solver(input: str) -> str:
    if not re.search(r"x(\^2|²)?", input, re.IGNORECASE):
        return "Error: No variable 'x' or quadratic term detected. Use MathExpressionCalculator for pure math expressions."
    input = input.lower()
    input = re.sub(r"[^\dx\^+\-\.=²]", "", input)
    input = input.replace("=0", "")
    try:
        match = re.search(r"([-+]?\d*\.?\d*)x(?:\^2|²)?([-+]\d*\.?\d*)x([-+]\d*\.?\d*)", input)
        if not match:
            return "Error: No valid quadratic equation found. Please provide input like '2x^2 + 3x + 1'."
        a, b, c = map(parse_coefficients, match.groups())
        if a == 0:
            return "Error: 'a' cannot be zero in a quadratic equation."
        discriminant = b**2 - 4*a*c
        if discriminant < 0:
            return "Error: No real roots exist for the given quadratic equation."
        elif discriminant == 0:
            root = -b / (2*a)
            return f"One real root exists: {root:.2f}"
        else:
            root1 = (-b + math.sqrt(discriminant)) / (2*a)
            root2 = (-b - math.sqrt(discriminant)) / (2*a)
            return f"Two real roots exist: {root1:.2f} and {root2:.2f}"
    except ValueError as e:
        return f"Error: Invalid input for quadratic equation. {str(e)}"
    
tools = [
    StructuredTool.from_function(
        name="MathExpressionCalculator",
        func=math_expression_calculator,
        description=(
            "Safely evaluate math expressions with only numbers and operators, like '(8 + 2) * (5 - 3) / 2'. "
            "DO NOT use if the input contains any 'x', 'x^2', 'x²', or is an equation to solve. "
            "DO NOT use for dates or temperature conversions."
        ),
        return_direct=True,
    ),
    StructuredTool.from_function(
        name="DaysPassedCalculator",
        func=days_passed_calculation,
        description=(
            "Use ONLY for questions like 'how many days have passed since 1995-01-01', or date difference questions. "
            "Input must contain a date in 'YYYY-MM-DD' format. "
            "DO NOT use for math expressions, temperature conversions, or equations with x."
        ),
        return_direct=True,
    ),
    StructuredTool.from_function(
        name="ConversionFtoC",
        func=fahrenheit_to_celsius,
        description=(
            "Convert Fahrenheit to Celsius. Input must be a valid number representing temperature in Fahrenheit. "
            "DO NOT use for math expressions, dates, or equations with x."
        ),
        return_direct=True,
    ),
    StructuredTool.from_function(
        name="QuadraticEquationSolver",
        func=quadratic_equation_solver,
        description=(
            "ONLY use if the user asks to solve for x or mentions a quadratic equation (with 'x', 'x^2', or 'x²'). "
            "DO NOT use for general math expressions, dates, or temperature conversions."
        ),
        return_direct=True,
    )
]

# --- Prompt Template ---
prompt = PromptTemplate(
    input_variables=["input"],
    template=(
        "You are a helpful and precise AI assistant.\n\n"
        "You must only answer with a single tool call per input.\n"
        "Never invent, repeat, or hallucinate tool calls. Only use the tool that matches the input.\n"
        "If the input contains 'x', 'x^2', 'x²', or is about solving for x, ONLY use the QuadraticEquationSolver.\n"
        "If the input is a math expression with only numbers and operators, use ONLY the MathExpressionCalculator.\n"
        "If the input asks about days since a date in 'YYYY-MM-DD' format, use ONLY the DaysPassedCalculator.\n"
        "If the input is about converting Fahrenheit to Celsius, use ONLY the ConversionFtoC.\n\n"
        "NEVER use MathExpressionCalculator for dates, temperatures, or equations with x.\n"
        "NEVER use DaysPassedCalculator or ConversionFtoC for general math or equations.\n"
        "NEVER use ConversionFtoC for dates or expressions with x.\n"
        "NEVER use QuadraticEquationSolver for general math, dates, or temperature conversions.\n\n"
        "When using ConversionFtoC, pass only the number (e.g., 98.6) and ignore any text.\n"
        "When using DaysPassedCalculator, pass the full input string so the tool can extract the date.\n"
        "When using MathExpressionCalculator, extract and pass ONLY the math expression (e.g., (8 + 2) * (5 - 3) / 2), not any surrounding text.\n"
        "When using QuadraticEquationSolver, pass the full equation string.\n\n"
        "Input: {input}\n\n"
        "{agent_scratchpad}"
    )
)

# --- LLM and Agent Setup ---
llm = ChatOllama(model="llama3.2:1b", temperature=0.0)  # Using Ollama's Llama 3.2 model
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)  # Uncomment to use OpenAI's GPT-3.5-turbo model
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1,
)

# --- Running the Agent ---
math_expression1 = executor.invoke({"input": "Compute the result of: (10 / 2) + 4 * (1 + 1)"})
print("Expression 1: ", math_expression1["output"])

math_expression2 = executor.invoke({"input": "Calculate the result of: (8 + 2) * (5 - 3) / 2"})
print("Expression 2: ", math_expression2["output"])

days_result = executor.invoke({"input": "How many days have passed since 1995-01-01?"})
print(days_result["output"])

f_to_celsius_result = executor.invoke({"input": "Convert 98.6 to celsius."})
print(f_to_celsius_result["output"])

quadratic_equation_result = executor.invoke({"input": "Solve the quadratic equation 2x^2 + 3x + 1 = 0"})
print(quadratic_equation_result["output"])
